In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

beta = 0.000275 #original: 0.02
beta_array = []
avg_error_func_array = []
sd_error_func_array = []

while beta < 0.01:
    xi_0 = 0.175
    T_1_ = 5
    xi_P = 0.175
    A_P = 0.01
    omega_P = 0.7
    A_0 = 0.5
    omega_T_ = 0.7
    beta = beta + 0.000025
    beta_array.append(beta)
    alpha = 0.02 #original: 0.02
    c = 0.1
    b_0 = 0.6
    a = 0.7
    T_2_ = 5
    g = 0.1


    tStart = 0
    tEnd = 2000

    X0 = [0.2, 0.1, 0.02, 0.01, 0.001]
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))

    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]

    # average and standard deviation of error function
    sum_error_func = 0
    i = 0
    while i<len(t_sol):
        sum_error_func += math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2)
        i += 1
    avg_error_func = sum_error_func/len(t_sol)
    avg_error_func_array.append(avg_error_func)
    i_sd = 0
    sum_difference_squared = 0
    while i_sd<len(t_sol):
        sum_difference_squared += (math.sqrt((x1_sol[i_sd]-x2_sol[i_sd])**2+(y1_sol[i_sd]-y2_sol[i_sd])**2)-avg_error_func)**2
        i_sd += 1
    sd_error_func = math.sqrt(sum_difference_squared/len(t_sol))
    sd_error_func_array.append(sd_error_func)

figure(figsize=(30, 5))
plt.plot(beta_array, avg_error_func_array)
plt.xlabel(''r'$\beta$', fontsize=15)
plt.ylabel(''r'$\theta$ average', fontsize=15)
plt.title('average of error function over time with respect to 'r'$\beta$', fontsize=15)
plt.show()

figure(figsize=(30, 5))
plt.plot(beta_array, sd_error_func_array)
plt.xlabel(''r'$\beta$', fontsize=15)
plt.ylabel(''r'$\theta$ standard deviation', fontsize=15)
plt.title('standard deviation of error function over time with respect to 'r'$\beta$', fontsize=15)
plt.show()